In [1]:
import paddle
from paddle.fluid.layers.nn import scale
from models import HSU_Net
from utils import *

In [2]:
paddle.set_device('cpu')
num_classes = 1
network = HSU_Net()
model = paddle.Model(network)
model.summary((-1, 3, 512, 512))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1      [[1, 3, 512, 512]]   [1, 40, 512, 512]         160      
 BatchNorm2D-1  [[1, 40, 512, 512]]   [1, 40, 512, 512]         160      
    ReLU-1      [[1, 40, 512, 512]]   [1, 40, 512, 512]          0       
   HSBlock-1    [[1, 40, 512, 512]]   [1, 40, 512, 512]          0       
 BatchNorm2D-2  [[1, 40, 512, 512]]   [1, 40, 512, 512]         160      
    ReLU-2      [[1, 40, 512, 512]]   [1, 40, 512, 512]          0       
   Conv2D-2     [[1, 40, 512, 512]]   [1, 64, 512, 512]        2,624     
 BatchNorm2D-3  [[1, 64, 512, 512]]   [1, 64, 512, 512]         256      
   Conv2D-3      [[1, 3, 512, 512]]   [1, 64, 512, 512]         256      
 BatchNorm2D-4  [[1, 64, 512, 512]]   [1, 64, 512, 512]         256      
HSBottleNeck-1   [[1, 3, 512, 512]]   [1, 64, 512, 512]          0       
  MaxPool2D-1    [[1, 512, 64, 64]] 

{'total_params': 7930337, 'trainable_params': 7900065}

In [3]:
train_dataset = SegDataset("G:\TJUZQC\code\python\PathologySegmentation\Training\pytorch\data\WSI\imgs",
                           "G:\TJUZQC\code\python\PathologySegmentation\Training\pytorch\data\WSI\masks", scale=0.5) # 训练数据集
val_dataset = SegDataset("G:\TJUZQC\code\python\PathologySegmentation\Training\pytorch\data\WSI\imgs",
                         "G:\TJUZQC\code\python\PathologySegmentation\Training\pytorch\data\WSI\masks", train=False, scale=0.5) # 验证数据集

<>:1: DeprecationWarning: invalid escape sequence \T
<>:2: DeprecationWarning: invalid escape sequence \T
<>:3: DeprecationWarning: invalid escape sequence \T
<>:4: DeprecationWarning: invalid escape sequence \T
<>:1: DeprecationWarning: invalid escape sequence \T
<>:2: DeprecationWarning: invalid escape sequence \T
<>:3: DeprecationWarning: invalid escape sequence \T
<>:4: DeprecationWarning: invalid escape sequence \T
<>:1: DeprecationWarning: invalid escape sequence \T
<>:2: DeprecationWarning: invalid escape sequence \T
<>:3: DeprecationWarning: invalid escape sequence \T
<>:4: DeprecationWarning: invalid escape sequence \T
<ipython-input-3-6b4dcd18618b>:1: DeprecationWarning: invalid escape sequence \T
  train_dataset = SegDataset("G:\TJUZQC\code\python\PathologySegmentation\Training\pytorch\data\WSI\imgs",
<ipython-input-3-6b4dcd18618b>:2: DeprecationWarning: invalid escape sequence \T
  "G:\TJUZQC\code\python\PathologySegmentation\Training\pytorch\data\WSI\masks", scale=0.5) # 训

In [4]:
train_loader = paddle.io.DataLoader(train_dataset, places=paddle.CPUPlace(), batch_size=1, shuffle=True)
val_loader = paddle.io.DataLoader(val_dataset, places=paddle.CPUPlace(), batch_size=1, shuffle=False)
network.train()

In [5]:
optim = paddle.optimizer.RMSProp(learning_rate=0.001,
                                 rho=0.9,
                                 momentum=0.0,
                                 epsilon=1e-07,
                                 centered=False,
                                 parameters=model.parameters())
loss_func = paddle.nn.BCEWithLogitsLoss()

In [19]:
for batch_id, data in enumerate(train_loader()):
    x_data = data[0]            # 训练数据
    y_data = data[1]            # 训练数据标签
    x_data = x_data.astype('float32')
    y_data = y_data.astype('float32')
    print(y_data.shape, y_data.dtype)
    predicts = network(x_data)    # 预测结果
    predicts = paddle.nn.Sigmoid()(predicts)
    pre_np = predicts.numpy()
    pre_np[pre_np > 0.5] = 1
    predicts = paddle.to_tensor(pre_np, 'int32')
    miou = paddle.metric.mean_iou(predicts, y_data.astype('int32'), 1)
    print(miou)
    break

[1, 1, 512, 512] VarType.FP32
(Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=True,
       [0.19058473]), Tensor(shape=[1], dtype=int32, place=CPUPlace, stop_gradient=True,
       [141693]), Tensor(shape=[1], dtype=int32, place=CPUPlace, stop_gradient=True,
       [33363]))
